In [1]:
import cv2
import mediapipe as mp
import pygame
import time

# Mediapipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Pygame setup
pygame.init()
pygame.mixer.init()

pygame.mixer.music.load("music.mp3")  

is_playing = False  
is_paused = False  
last_action_time = 0 
gesture_cooldown = 1.5  


def recognize_gesture(hand_landmarks):

    thumb_tip = hand_landmarks[4]
    index_tip = hand_landmarks[8]
    middle_tip = hand_landmarks[12]
    ring_tip = hand_landmarks[16]
    pinky_tip = hand_landmarks[20]

   
    index_mid = hand_landmarks[6]
    middle_mid = hand_landmarks[10]
    ring_mid = hand_landmarks[14]
    pinky_mid = hand_landmarks[18]

    wrist = hand_landmarks[0]

    index_curled = index_tip.y > index_mid.y 
    middle_curled = middle_tip.y > middle_mid.y 
    ring_curled = ring_tip.y > ring_mid.y  
    pinky_curled = pinky_tip.y > pinky_mid.y  

    fingers_curled = index_curled and middle_curled and ring_curled and pinky_curled

    thumb_extended = thumb_tip.y < wrist.y and all(thumb_tip.y < finger.y for finger in [index_tip, middle_tip, ring_tip, pinky_tip])

    if thumb_extended and fingers_curled:
        return "thumbs_up"

    fingers_extended = all(finger.y < wrist.y for finger in [index_tip, middle_tip, ring_tip, pinky_tip])
    if fingers_extended:
        return "palm"

    return "none"


def handle_gesture(gesture, current_time):

    global is_playing, is_paused, last_action_time

    if gesture == "thumbs_up" and current_time - last_action_time > gesture_cooldown:
        if not is_playing:  
            print("Playing music...")
            pygame.mixer.music.play(-1)  
            is_playing = True
            is_paused = False
        elif is_paused:  
            print("Resuming music...")
            pygame.mixer.music.unpause()
            is_paused = False
        last_action_time = current_time

    elif gesture == "palm" and current_time - last_action_time > gesture_cooldown:
        if is_playing and not is_paused:  
            print("Pausing music...")
            pygame.mixer.music.pause()
            is_paused = True
        else:
            print("Music is already paused.")
        last_action_time = current_time


cap = cv2.VideoCapture(0)
print("Show a thumbs up to play/resume, show your palm to pause. Press 'q' to quit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture video. Exiting...")
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            gesture = recognize_gesture(hand_landmarks.landmark)
            current_time = time.time()

            handle_gesture(gesture, current_time)

            cv2.putText(
                frame,
                f"Gesture: {gesture}",
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0),
                2,
            )

    cv2.imshow("Hand Gesture Music Control", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()
hands.close()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
Show a thumbs up to play/resume, show your palm to pause. Press 'q' to quit.
Playing music...
Pausing music...
Resuming music...
Pausing music...
Resuming music...


KeyboardInterrupt: 